In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
#第二題

import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.models import Sequential, load_model

# 忽略警告
import warnings
warnings.filterwarnings("ignore")

# 初始化圖像和標籤列表
images = []
labels = []

# 初始化驗證集的資料
val_images_accumulated = []
val_labels_accumulated = []

# 刪除檔案
# 設置模型保存路徑
model_path = 'saved_model.h5'
if os.path.exists(model_path):
    os.remove(model_path)
    print(f"檔案 '{model_path}' 已成功刪除。")
else:
    print(f"檔案 '{model_path}' 不存在，無需刪除。")
    



# 檢查是否已經存在保存的模型，如果是則加載它
if os.path.exists(model_path):
    model = load_model(model_path)
else:
    # 如果模型不存在，則創建一個新的模型
     model = tf.keras.Sequential([
            # 第一层卷积层
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            # 第二层卷积层
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            # 第二层卷积层
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),

            # 扁平化层
            tf.keras.layers.Flatten(),
            # 全连接层

            tf.keras.layers.Dense(10, activation='softmax')
        ])

    

# 讀取標籤資料
for filename in os.listdir('/kaggle/input/advancedml-hw2/hw2/HW2_MNIST_train'):
    if filename.endswith('.txt'):
        labels_file_path = os.path.join('/kaggle/input/advancedml-hw2/hw2/HW2_MNIST_train', filename)
        with open(labels_file_path, 'r') as file:
            lines = file.readlines()

        # 處理每張圖像
        for line in lines:
            parts = line.strip().split('\t')
            digit = int(parts[0])
            image_path = os.path.join('/kaggle/input/advancedml-hw2/hw2/HW2_MNIST_train', filename.replace('.txt', '.png'))
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (64, 64))
            if img is None:
                print(f"Error reading image: {image_path}")
                continue
            digit_img = img.astype(np.float32) / 255.0
            images.append(digit_img)
            labels.append(digit)

            # 如果已經收集到1000張圖片，則執行訓練
            if len(images) == 1000:

                # 將列表轉換為NumPy數組
                images_np = np.array(images)
                labels_np = np.array(labels)

                # 將標籤進行獨熱編碼
                labels_one_hot = to_categorical(labels_np, num_classes=10)

                # 加載模型
                if os.path.exists(model_path):
                    model = load_model(model_path)

                # 將數據集劃分為訓練集和驗證集7:3
                train_images, val_images, train_labels, val_labels = train_test_split(images_np, labels_one_hot, test_size=0.3, random_state=42)

                # 累積驗證集的資料
                val_images_accumulated.extend(val_images)
                val_labels_accumulated.extend(val_labels)
                num_val_images = len(val_images_accumulated)

                # 編譯模型
                model.compile(optimizer='adam',
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])

                # 訓練模型
                model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_data=(np.array(val_images_accumulated), np.array(val_labels_accumulated)), verbose=0)

                # 保存訓練好的模型
                model.save(model_path)

                # 清空資料
                images = []
                labels = []

print('模型訓練完成')



#計算第二題所要的資訊

num_val_images = len(val_images_accumulated)
#print("驗證集中的圖片數量:", num_val_images)

# 將 val_images_accumulated 中的圖像形狀從 (900, 64, 64) 轉換為 (900, 64, 64, 1)
val_images_accumulated1 = np.expand_dims(val_images_accumulated, axis=-1)

# 使用模型預測
val_pred = model.predict(val_images_accumulated1, verbose=0)
val_pred_labels = np.argmax(val_pred, axis=1)
val_true_labels = np.argmax(val_labels_accumulated, axis=1)
val_conf_matrix = confusion_matrix(val_true_labels, val_pred_labels)

# 驗證集Top-1
val_top1_acc = accuracy_score(val_true_labels, val_pred_labels)
val_top1_num = np.sum(val_true_labels == val_pred_labels)
val_top1_denom = len(val_true_labels)

print("Top-1 Accuracy:", val_top1_acc)
print("Top-1 分子:", val_top1_num)
print("Top-1 分母:", val_top1_denom)


print("\n2.")
# 計算模型的層數
num_layers = len(model.layers)
print("\n(1)Number of layers:", num_layers)

# 獲取模型參數數量
num_params = model.count_params()
print("\n(2)Number of parameters:", num_params)

# 計算測試集Top-1準確率
value = val_top1_acc / num_params
print("\n(3)Top-1 Accuracy / # of parameter=",val_top1_acc,"/" , num_params,"=",value)

檔案 'saved_model.h5' 不存在，無需刪除。
模型訓練完成
Top-1 Accuracy: 0.8574914089347079
Top-1 分子: 24953
Top-1 分母: 29100

2.

(1)Number of layers: 22

(2)Number of parameters: 19946

(3)Top-1 Accuracy / # of parameter= 0.8574914089347079 / 19946 = 4.299064518874501e-05
